<a href="https://colab.research.google.com/github/sh3986/workplace/blob/master/main_v1_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Library

In [95]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:.5f}".format

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras import optimizers
tf.__version__

'2.8.0'

In [3]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Kaggle/project_2_cnn"

In [4]:
import os
os.chdir(data_path)

In [5]:
data = []
labels = []

height = 30
width = 30
channels = 3
num_classes = 43

In [ ]:
# for i in range(num_classes) :
#     path = f'./data/Train/{i}/'
#     Class = os.listdir(path)
#     for a in Class:
#         image = cv2.imread(path + a)
#         image_from_array = Image.fromarray(image, 'RGB')
#         size_image = image_from_array.resize((height, width))
#         data.append(np.array(size_image))
#         labels.append(i)

# data = np.array(data)
# labels = np.array(labels)

In [ ]:
# import pickle
# with open("/content/drive/MyDrive/Colab Notebooks/Kaggle/project_2_cnn/data/train_data.pkl","wb") as f:
#     pickle.dump(data, f)

In [ ]:
# with open("/content/drive/MyDrive/Colab Notebooks/Kaggle/project_2_cnn/data/train_label.pkl","wb") as f:
    # pickle.dump(labels, f)

In [6]:
import pickle 

with open('./data/train_data.pkl', 'rb') as f:
    data = pickle.load(f)

with open('./data/train_label.pkl', 'rb') as f:
    labels = pickle.load(f)

### 전처리

In [11]:
print(data.shape)
print(labels.shape)

(26010, 30, 30, 3)
(26010,)


In [ ]:
# for a in Class:
#     image = cv2.imread(path + a)
#     image_from_array = Image.fromarray(image, 'RGB')
#     size_image = image_from_array.resize((height, width))
#     data.append(np.array(size_image))
#     labels.append(i)

In [115]:
X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.2, random_state=42, stratify = labels)

In [116]:
X_train, X_test = X_train/255.0, X_valid/255.0

In [117]:
y_train = tf.stop_gradient(tf.one_hot(y_train, num_classes))
y_valid_one_hot = tf.one_hot(y_valid, num_classes)

### 모델 생성

In [118]:
model = Sequential()

model.add(Input(shape=(30,30,3)))
model.add(Conv2D(32, 5, [1,1], activation='relu', padding='same'))
model.add(Conv2D(64, 5, [1,1], activation='relu', padding='same'))
model.add(MaxPool2D((2,2), (2,2), padding='same'))
model.add(Conv2D(128, 5, [1,1], activation='relu', padding='same'))
model.add(Conv2D(64, 5, [1,1], activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# model = models.Sequential()
# model.add(layers.Flatten(input_shape=(4,4,512)))
# model.add(layers.Dense(256, activation='relu', input_dim=(4*4*512)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(num_classes, activation='softmax'))
# model.summary()

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 30, 30, 32)        2432      
                                                                 
 conv2d_47 (Conv2D)          (None, 30, 30, 64)        51264     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 15, 15, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (None, 15, 15, 128)       204928    
                                                                 
 conv2d_49 (Conv2D)          (None, 15, 15, 64)        204864    
                                                                 
 flatten_15 (Flatten)        (None, 14400)             0         
                                                     

In [119]:
model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

### 모델 학습

In [120]:
model.fit(X_train, y_train, batch_size=1024, epochs=20, shuffle=True, validation_data=(X_valid, y_valid_one_hot))

Epoch 1/20
21/21 [==============================] - 2s 59ms/step - loss: 3.5729 - acc: 0.0559 - val_loss: 97.5782 - val_acc: 0.0815
Epoch 2/20
21/21 [==============================] - 1s 51ms/step - loss: 3.0198 - acc: 0.1820 - val_loss: 152.7282 - val_acc: 0.2832
Epoch 3/20
21/21 [==============================] - 1s 51ms/step - loss: 1.7856 - acc: 0.4724 - val_loss: 103.0725 - val_acc: 0.5925
Epoch 4/20
21/21 [==============================] - 1s 51ms/step - loss: 0.8144 - acc: 0.7487 - val_loss: 53.2248 - val_acc: 0.7449
Epoch 5/20
21/21 [==============================] - 1s 51ms/step - loss: 0.3986 - acc: 0.8776 - val_loss: 53.1720 - val_acc: 0.8093
Epoch 6/20
21/21 [==============================] - 1s 51ms/step - loss: 0.2202 - acc: 0.9342 - val_loss: 44.9235 - val_acc: 0.8260
Epoch 7/20
21/21 [==============================] - 1s 51ms/step - loss: 0.1273 - acc: 0.9645 - val_loss: 32.0629 - val_acc: 0.8695
Epoch 8/20
21/21 [==============================] - 1s 51ms/step - loss: 0

### 모델 테스트

In [121]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_valid)
y_pred = tf.argmax(y_pred, axis=1).numpy()

In [122]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83        36
           1       0.78      0.99      0.88       252
           2       0.81      0.97      0.88       252
           3       0.93      0.90      0.91       252
           4       0.93      0.95      0.94       252
           5       0.97      0.68      0.80       252
           6       1.00      1.00      1.00        36
           7       0.91      0.88      0.90       252
           8       0.99      0.73      0.84       252
           9       0.96      0.95      0.96       252
          10       0.98      0.94      0.96       252
          11       0.93      0.96      0.95       252
          12       0.97      1.00      0.98       252
          13       0.96      1.00      0.98       252
          14       0.98      1.00      0.99       126
          15       0.79      0.98      0.88       126
          16       1.00      0.94      0.97        36
          17       1.00    

In [123]:
print(confusion_matrix(y_valid, y_pred, labels=range(9)))

[[ 36   0   0   0   0   0   0   0   0]
 [  0 250   1   0   1   0   0   0   0]
 [  0   6 244   1   0   0   0   0   0]
 [  0   3  10 226   0   2   0   0   0]
 [  3   7   0   0 240   0   0   0   0]
 [  1   7  40   4   0 171   0  11   1]
 [  0   0   0   0   0   0  36   0   0]
 [  0  13   5   0   2   0   0 222   1]
 [ 11  23   0   0  10   2   0   8 183]]


### 주석